# ROB701@MBZUAI Fall 2025 - Lab 1009

**TA:** Tianyu Liu  
**Email:** [Tianyu.Liu@mbzuai.ac.ae](mailto:Tianyu.Liu@mbzuai.ac.ae)

**Topic:** Dynamics

Note: Code sections pending implementation are labeled “[To Be Implemented]”.

<img src="https://raw.githubusercontent.com/RealGaule/ROB701_Lab/842ab4e9851bc4fcc03271e134283e67db776383/docs/Robotics.png" width="320">

## 0.Three-Link Planar Robot
Suppose we have a 3-link planar robot with three revolute joints (3R). The following code defines a parameter class for this robot.

`Parameters` (class)

**Purpose:** Collects the physical properties of the 3R planar manipulator (all scalars, SI units).

**Fields & Physical Meaning**
- `m1`, `m2`, `m3` *(kg)* — mass of link 1/2/3.
- `Iz1`, `Iz2`, `Iz3` *(kg·m²)* — moment of inertia of each link about its joint axis (out of plane).
- `l1`, `l2`, `l3` *(m)* — geometric length of link 1/2/3.
- `g` *(m/s²)* — gravitational acceleration (default `9.81`).

**Notes**
- All angles are in **radians**; angular rates in **rad/s**; angular accelerations in **rad/s²**.
- Joint axes are assumed at inter-link connection points; the mechanism is planar with 3 revolute joints.
- The absolute reference of gravitational potential is conventional; only differences matter for energy conservation checks.

In [ ]:
import numpy as np
from scipy.integrate import odeint
from matplotlib import pyplot as plt
from matplotlib import animation
from dataclasses import dataclass
from IPython.display import HTML

@dataclass
class Parameters:
    m1: float = 1.0   # [kg]
    m2: float = 1.0   # [kg]
    m3: float = 0.2   # [kg]
    Iz1: float = 0.1  # [kg·m^2]
    Iz2: float = 0.1  # [kg·m^2]
    Iz3: float = 0.02 # [kg·m^2]
    l1: float = 1.0   # [m]
    l2: float = 1.0   # [m]
    l3: float = 0.25  # [m]
    g: float = 9.81   # [m/s^2]

## 1. Lagrange Dynamics

`energy(t, z, p)` function

**Purpose:** Compute kinetic energy (KE), gravitational potential energy (PE), and total energy (TE) at each time step.

**Signature**
- Inputs:
  - `t`: `(m,) ndarray` — time grid in seconds. Used for alignment (not directly in formulas). Must satisfy `len(t) == z.shape[0]`.
  - `z`: `(m, 6) ndarray` — state trajectory; each row is `[theta1, omega1, theta2, omega2, theta3, omega3]`.
    - `theta*` in *rad*, `omega*` in *rad/s*.
  - `p`: `Parameters` — physical parameters (m, kg, kg·m², m/s²).
- Outputs:
  - `KE`: `(m,) ndarray` — kinetic energy in *J*.
  - `PE`: `(m,) ndarray` — gravitational potential energy in *J*.
  - `TE`: `(m,) ndarray` — total energy in *J* = `KE + PE`.

**Physical meaning**
- KE includes link self-rotation terms and coupling cross-terms due to the mechanism.
- PE is computed from link CoM heights times `g` (reference level is conventional).

**Notes**
- Expect small numerical drift in `TE` due to integration errors; that is normal.
- Units must be consistent (meters, kilograms, seconds, radians).


In [ ]:
def energy(t: np.ndarray, z: np.ndarray, p: Parameters):
    assert z.ndim == 2 and z.shape[1] == 6, "z must have shape (m, 6)"
    assert len(t) == z.shape[0], "len(t) must equal the number of rows in z"

    m1, m2, m3 = p.m1, p.m2, p.m3
    Iz1, Iz2, Iz3 = p.Iz1, p.Iz2, p.Iz3
    l1, l2, l3, g = p.l1, p.l2, p.l3, p.g

    KE_all, PE_all, TE_all = [], [], []

    # [To Be Implemented]

    return np.array(KE_all), np.array(PE_all), np.array(TE_all)



`dynamics(z, t, m1, m2, m3, Iz1, Iz2, Iz3, l1, l2, l3, g)` function

**Purpose:** Right-hand side of the first-order state dynamics $\dot z = f(z,t)$ derived from the Lagrange model  
$M(q)\,\ddot q + C(q,\dot q) + G(q) = 0$. We assemble `z = [q, qdot]` and return `[qdot, qddot]`.

**Signature**
- Inputs:
  - `z`: `(6,) ndarray` — current state `[theta1, omega1, theta2, omega2, theta3, omega3]` (rad, rad/s).
  - `t`: `float` — current time (s). The model is time-invariant; included for ODE solver compatibility.
  - `m1, m2, m3` *(kg)* — link masses.
  - `Iz1, Iz2, Iz3` *(kg·m²)* — link inertias about joint axes.
  - `l1, l2, l3` *(m)* — link lengths.
  - `g` *(m/s²)* — gravity.
- Outputs:
  - `dzdt`: `(6,) ndarray` — time derivative `[omega1, alpha1, omega2, alpha2, omega3, alpha3]` (rad/s, rad/s²).

**Notes**
- External torques/damping are **not** included here; to add control, include torques on the right-hand side with consistent sign.
- Numerical considerations: avoid forming `inv(M)` explicitly for performance/conditioning; here it is fine for small 3×3 systems.


In [ ]:


def dynamics(z: np.ndarray, t: float,
        m1: float, m2: float, m3: float,
        Iz1: float, Iz2: float, Iz3: float,
        l1: float, l2: float, l3: float, g: float) -> np.ndarray:
    theta1, omega1, theta2, omega2, theta3, omega3 = z

    # [To Be Implemented]

    return np.array([omega1, qdd[0], omega2, qdd[1], omega3, qdd[2]])

## 2. Simulation

Integrate the state ODE using `scipy.integrate.odeint`

In [ ]:
def simulate(t: np.ndarray, z0: np.ndarray, p: Parameters) -> np.ndarray:
    assert z0.shape == (6,), "z0 must have shape (6,)"
    args = (p.m1, p.m2, p.m3, p.Iz1, p.Iz2, p.Iz3, p.l1, p.l2, p.l3, p.g)
    z = odeint(dynamics, z0, t, args=args)
    return z

def animate_planar_3R(t: np.ndarray, z: np.ndarray, p: Parameters) -> animation.FuncAnimation:
    l1, l2, l3 = p.l1, p.l2, p.l3
    fig, ax = plt.subplots()
    ll = 1.1*(l1 + l2 + l3)
    ax.set_xlim(-ll, ll); ax.set_ylim(-ll, ll); ax.set_aspect('equal')
    ax.set_xlabel('x [m]'); ax.set_ylabel('y [m]')
    ax.set_title('Planar 3R Motion')

    # Lines and markers
    line1, = ax.plot([], [], lw=2)
    line2, = ax.plot([], [], lw=2)
    line3, = ax.plot([], [], lw=2)
    com1, = ax.plot([], [], 'o', markersize=6)
    com2, = ax.plot([], [], 'o', markersize=6)
    com3, = ax.plot([], [], 'o', markersize=6)

    def init():
        for artist in (line1, line2, line3):
            artist.set_data([], [])
        for pt in (com1, com2, com3):
            pt.set_data([], [])
        return line1, line2, line3, com1, com2, com3

    def frame(i):
        theta1, theta2, theta3 = z[i,0], z[i,2], z[i,4]
        O = np.array([0.0, 0.0])
        P = np.array([l1*np.cos(theta1), l1*np.sin(theta1)])
        Q = P + np.array([l2*np.cos(theta1+theta2), l2*np.sin(theta1+theta2)])
        E = Q + np.array([l3*np.cos(theta1+theta2+theta3), l3*np.sin(theta1+theta2+theta3)])
        G1 = np.array([0.5*l1*np.cos(theta1), 0.5*l1*np.sin(theta1)])
        G2 = P + np.array([0.5*l2*np.cos(theta1+theta2), 0.5*l2*np.sin(theta1+theta2)])
        G3 = Q + np.array([0.5*l3*np.cos(theta1+theta2+theta3), 0.5*l3*np.sin(theta1+theta2+theta3)])

        line1.set_data([O[0], P[0]], [O[1], P[1]])
        line2.set_data([P[0], Q[0]], [P[1], Q[1]])
        line3.set_data([Q[0], E[0]], [Q[1], E[1]])
        com1.set_data([G1[0]], [G1[1]])
        com2.set_data([G2[0]], [G2[1]])
        com3.set_data([G3[0]], [G3[1]])
        return line1, line2, line3, com1, com2, com3

    interval_ms = int(1000.0 * (t[1] - t[0]))
    ani = animation.FuncAnimation(fig, frame, init_func=init, frames=len(t), interval=interval_ms, blit=True)
    return ani

## 3. Evaluation

**simulate angles over time**
- Create a time grid and zero initial state.
- Integrate the dynamics to obtain `z(t)`.
- Plot joint angles vs. time.
- Output animation in GIF.


In [ ]:
p = Parameters()
t = np.linspace(0, 10, 201)   # (m,) seconds
z0 = np.zeros(6)              # (6,) initial state

z = simulate(t, z0, p)        # (m,6)

plt.figure()
plt.plot(t, z[:,0], label='theta1 [rad]')
plt.plot(t, z[:,2], label='theta2 [rad]')
plt.plot(t, z[:,4], label='theta3 [rad]')
plt.xlabel('t [s]'); plt.ylabel('angle [rad]'); plt.legend(); plt.grid(True); plt.show()

In [ ]:
ani = animate_planar_3R(t, z, p)
HTML(ani.to_jshtml())

**energy curves**

Compute and visualize `KE`, `PE`, and `TE` over time to sanity-check energy behavior (expect mild numerical drift only).


In [ ]:
KE, PE, TE = energy(t, z, p)
plt.figure()
plt.plot(t, KE, label='KE [J]')
plt.plot(t, PE, label='PE [J]')
plt.plot(t, TE, label='TE [J]')
plt.xlabel('t [s]'); plt.ylabel('Energy [J]'); plt.legend(); plt.grid(True); plt.show()